<a href="https://colab.research.google.com/github/njsuriya/ML_clf_algorithms/blob/main/Tag_Prediction_using_Multilabelclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 4.8 MB/s 


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/multilabel_ds.csv')

In [4]:
df.dtypes

title      object
tags       object
mysql       int64
python    float64
php       float64
dtype: object

In [5]:
df['mysql'] = df['mysql'].astype(float)

In [6]:
df.dtypes

title      object
tags       object
mysql     float64
python    float64
php       float64
dtype: object

In [7]:
df['title'] = df['title'].str.lower()

In [8]:
df.head()

,title,tags,mysql,python,php
0,flask-sqlalchemy - when are the tables/databas...,"['python', 'mysql']",1.0,1.0,0.0
1,combining two php variables for mysql query,"['php', 'mysql']",1.0,0.0,1.0
2,'counting' the number of records that match a ...,"['php', 'mysql']",1.0,0.0,1.0
3,insert new row in a table and auto id number. ...,"['php', 'mysql']",1.0,0.0,1.0
4,create multiple mysql tables using php,"['php', 'mysql']",1.0,0.0,1.0


In [9]:
import string 
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def remove_punc(text):
  return text.translate(str.maketrans('','',punct))

df['title'] = df['title'].apply(lambda text_line: remove_punc(text_line))

In [11]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def remove_stop(text):
  return " ".join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])

df['title'] = df['title'].apply(lambda text_line: remove_stop(text_line))

In [12]:
x = df['title']
y = df[['mysql','python','php']]

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tfidf = TfidfVectorizer()

In [14]:
x = tfidf.fit_transform(x).toarray()

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [17]:
df.head()

,title,tags,mysql,python,php
0,flasksqlalchemy tablesdatabases created destroyed,"['python', 'mysql']",1.0,1.0,0.0
1,combining php variables mysql query,"['php', 'mysql']",1.0,0.0,1.0
2,counting number records match certain criteria...,"['php', 'mysql']",1.0,0.0,1.0
3,insert new row table auto id number php mysql,"['php', 'mysql']",1.0,0.0,1.0
4,create multiple mysql tables using php,"['php', 'mysql']",1.0,0.0,1.0


In [18]:
# x_train = tfidf.fit_transform(x_train).toarray()

In [19]:
from skmultilearn.problem_transform import BinaryRelevance,LabelPowerset,ClassifierChain
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score

In [31]:
binary_clf = BinaryRelevance(MultinomialNB())
model = binary_clf.fit(x_train,y_train)
binary_clf_pred = model.predict(x_test)
print("Binary Relevance NB ::: Accuracy Score {} , F1 Score {}".format(accuracy_score(y_test,binary_clf_pred),f1_score(y_test,binary_clf_pred,average='weighted')))

Binary Relevance NB ::: Accuracy Score 0.75 , F1 Score 0.8741312741312741


In [30]:
binary_clf.classifiers_

[MultinomialNB(), MultinomialNB(), MultinomialNB()]

In [32]:
model.classifiers_

[MultinomialNB(), MultinomialNB(), MultinomialNB()]

In [22]:
label_clf = LabelPowerset(MultinomialNB())
label_clf.fit(x_train,y_train)
label_clf_pred = label_clf.predict(x_test)
print("LabelPowerset NB ::: Accuracy Score {} , F1 Score {}".format(accuracy_score(y_test,label_clf_pred),f1_score(y_test,label_clf_pred,average='weighted')))

LabelPowerset NB ::: Accuracy Score 0.75 , F1 Score 0.8741312741312741


In [23]:
chain_clf = ClassifierChain(MultinomialNB())
chain_clf.fit(x_train,y_train)
chain_clf_pred = label_clf.predict(x_test)
print("Classifier Chain NB ::: Accuracy Score {} , F1 Score {}".format(accuracy_score(y_test,chain_clf_pred),f1_score(y_test,chain_clf_pred,average='weighted')))

Classifier Chain NB ::: Accuracy Score 0.75 , F1 Score 0.8741312741312741


In [36]:
chain_clf.classifiers_

[MultinomialNB(), MultinomialNB(), MultinomialNB()]

In [43]:
test_ques = [' unable to run python file ']

binary_clf.predict(tfidf.transform(test_ques)).toarray()

array([[1., 1., 0.]])